## Import Necessary Libraries

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Vizualize one data sample

In [2]:
#path to sample:
Kitti_file='/media/i2r/My Passport/KITTI/object/sampledata/velodyne/000000.bin'
annotation_file='/media/i2r/My Passport/KITTI/object/sampledata/label_2/000000.txt'
calibration_file='/media/i2r/My Passport/KITTI/object/sampledata/calib/000000.txt'
# Load binary point cloud
bin_pcd = np.fromfile(Kitti_file, dtype=np.float32)
point_cloud_range = [0, -39.68, -3, 69.12, 39.68, 1]

In [3]:
# Reshape and drop intensity values
points = bin_pcd.reshape((-1, 4))[:, 0:4]
np_pcd=[]
for point in points:
    if point[1]>-39.68 and point[1]<39.68:
        if point[0]>0 and point[0]<69.12:
            if point[2]>-3 and point[2]<1:
                np_pcd.append(list(point[0:3]))

In [4]:
np_pcd = np.asarray(np_pcd)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(np_pcd)
o3d.visualization.draw_geometries([pcd])

## Read annotation file

### Some relevant functions

In [5]:
#h- height, w - width, l - length, x,y,z - locations of center and yaw is rotation around Y axis; 
#more details here: https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html

In [6]:
#Read the annotation file
def read_annotation_file(annotation_file):
    annotations = []
    with open(annotation_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
          #Ped for pedestrian, Car for car etc.
          if line[0:3]=='Ped':
            data = line.strip().split(' ')
            # Extract the relevant information from the line
            h, w, l, x, y, z, yaw = map(float, data[8:15])
            annotation = [h, w, l, x, y, z, yaw]
            annotations.append(annotation)
    return annotations

These functions are not important for you to spend time on. Just know that the annotations in Kitti are in camera coordinates and these functions are used to turn them into LiDAR coordinates - to work with the data and visualize it easier. You can simply ignore them.

In [7]:
def roty(t):
    """ Rotation about the y-axis. """
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])

In [8]:
def project_rect_to_velo( pts_3d_rect):
        """ Input: nx3 points in rect camera coord.
            Output: nx3 points in velodyne coord.
        """
        pts_3d_ref = project_rect_to_ref(pts_3d_rect)
        return project_ref_to_velo(pts_3d_ref)

In [9]:
def project_rect_to_ref(pts_3d_rect):
        """ Input and Output are nx3 points """
        return np.transpose(np.dot(np.linalg.inv(R0), np.transpose(pts_3d_rect)))

In [10]:
def read_calib_file( filepath):
        """ Read in a calibration file and parse into a dictionary.
        Ref: https://github.com/utiasSTARS/pykitti/blob/master/pykitti/utils.py
        """
        data = {}
        with open(filepath, "r") as f:
            for line in f.readlines():
                line = line.rstrip()
                if len(line) == 0:
                    continue
                key, value = line.split(":", 1)
                # The only non-float values in these files are dates, which
                # we don't care about anyway
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass

        return data

In [11]:
def inverse_rigid_trans(Tr):
    """ Inverse a rigid body transform matrix (3x4 as [R|t])
        [R'|-R't; 0|1]
    """
    inv_Tr = np.zeros_like(Tr)  # 3x4
    inv_Tr[0:3, 0:3] = np.transpose(Tr[0:3, 0:3])
    inv_Tr[0:3, 3] = np.dot(-np.transpose(Tr[0:3, 0:3]), Tr[0:3, 3])
    return inv_Tr

In [12]:
def project_ref_to_velo( pts_3d_ref):
    pts_3d_ref = cart2hom(pts_3d_ref)  # nx4
    return np.dot(pts_3d_ref, np.transpose(C2V))

In [13]:
def cart2hom(pts_3d):
    """ Input: nx3 points in Cartesian
            Oupput: nx4 points in Homogeneous by pending 1
    """
    n = pts_3d.shape[0]
    pts_3d_hom = np.hstack((pts_3d, np.ones((n, 1))))
    return pts_3d_hom

### This function is to obtain the 8 corners of the annotation box

In [14]:
def compute_box_3d(annotation, P):
    """ Takes an object and a projection matrix (P) and projects the 3d
        bounding box into the image plane.
        Returns:
            corners_2d: (8,2) array in left image coord.
            corners_3d: (8,3) array in in rect camera coord.
    """
    # compute rotational matrix around yaw axis
    R = roty(annotation[6])

    # 3d bounding box dimensions
    l = annotation[2]
    w = annotation[1]
    h = annotation[0]

    # 3d bounding box corners
    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
    y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

    # rotate and translate 3d bounding box
    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
    # print corners_3d.shape
    corners_3d[0, :] = corners_3d[0, :] + annotation[3]
    corners_3d[1, :] = corners_3d[1, :] + annotation[4]
    corners_3d[2, :] = corners_3d[2, :] + annotation[5]
    return np.transpose(corners_3d)

### Get the annotation and calibration files

In [15]:
annotations = read_annotation_file(annotation_file)
calibs = read_calib_file(calibration_file)

### Apply the necessary transformations to get the LiDAR coordinates

In [16]:
R0 = calibs["R0_rect"]
R0 = np.reshape(R0, [3, 3])
V2C = calibs["Tr_velo_to_cam"]
V2C = np.reshape(V2C, [3, 4])
C2V = inverse_rigid_trans(V2C)
P = calibs["P2"]
P = np.reshape(P, [3, 4])  

### Obtain the box coordinates of the annotation

In [17]:
annots=[]
for annotation in annotations:
    box3d_pts_3d = compute_box_3d(annotation, P)
    pts_3d_ref=np.transpose(np.dot(np.linalg.inv(R0), np.transpose(box3d_pts_3d)))
    pts_3d_ref = cart2hom(pts_3d_ref) 
    box3d_pts_3d_velo=np.dot(pts_3d_ref, np.transpose(C2V))
    annots.append(box3d_pts_3d_velo)

### Draw the box in 3D and vizualize it on the scene

In [19]:
line_sets=[]
# Our lines span from points 0 to 1, 1 to 2, 2 to 3, etc...
lines = [[0, 1], [1, 2], [2, 3], [0, 3],
         [4, 5], [5, 6], [6, 7], [4, 7],
         [0, 4], [1, 5], [2, 6], [3, 7]]

# Use the same color for all lines
colors = [[1, 0, 0] for _ in range(len(lines))]
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(box3d_pts_3d_velo)
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.colors = o3d.utility.Vector3dVector(colors)

#Visualize
o3d.visualization.draw_geometries([line_set,pcd])